In [2]:
import sys
sys.path.append('/Users/chrisolen/Documents/uchicago_courses/optimization/project/urban-demand-allocation')

import pandas as pd
import numpy as np
import json

from py2neo import Graph

from sklearn.linear_model import LinearRegression

import utilities
from demand_models.build_demand_model_utils import business_filter, connect_to_neo4j, graph_to_demand_model

In [4]:
demand = pd.read_csv("demand_model.csv")
demand

,sales_volume_location,sales_volume_location_2016,sales_volume_location_2015,zestimate,primary_type,zestimate_NEXT_TO,primary_type_NEXT_TO
0,7959.0,12148.0,16198.0,369466.928527,2309.0,501768.966979,1732.833333
1,9640.0,13420.0,13420.0,345766.027933,2089.0,456699.347619,4845.750000
2,3853.0,6089.0,3000.0,174195.314741,9029.0,264464.323211,6675.200000
3,4287.0,6045.0,6045.0,194151.698144,10237.0,346094.527748,6511.166667
4,9063.0,12638.0,12638.0,550774.843931,519.0,817196.592154,4615.000000
...,...,...,...,...,...,...,...
772,13863.0,21757.0,8481.0,229517.369830,1992.0,192372.820101,1733.600000
773,12218.0,19176.0,19176.0,216874.451473,950.0,204954.406780,1449.000000
774,1410.0,2212.0,2212.0,161538.790567,4524.0,228346.399239,2853.666667
775,767.0,796.0,1327.0,802738.017279,9284.0,529645.318854,1800.666667


In [5]:
# least squares model for 'D' component (i.e. including past sales data) of objective function:

D_y = np.array(demand["sales_volume_location"])
D_features = demand[demand.columns[demand.columns != "sales_volume_location"]]
D_X = np.array(D_features)

# least squares model for 'L' component (i.e. excluding past sales data) of objective functon:
L_features = demand[[i for i in D_features.columns if "sales" not in i]]
L_y = D_y
L_X = np.array(L_features)





In [6]:
D_lr = LinearRegression()

D_model = D_lr.fit(D_X,D_y)

L_lr = LinearRegression()

L_model = L_lr.fit(L_X,L_y)


In [7]:
D_coef = list(D_model.coef_)
D_coef

[0.7402733259748637,
 0.1303118399846292,
 -0.0034529577130379546,
 -0.04075228681623255,
 0.0032423546186402807,
 -0.18018045401480864]

In [8]:
L_coef = list(L_model.coef_)
L_coef

[0.0015318111839094208,
 -0.2516841223603309,
 0.006086799257956077,
 -0.5301625724129638]

In [27]:
# save as txt file

D_model_params = [D_demand_features, D_coef]

with open('opt_variables/D_demand_coef.txt', 'w') as model_text:
    for listitem in D_model_params:
        model_text.write('%s\n' % listitem)
        
        # save as txt file

L_model_params = [L_demand_features, L_coef]

with open('opt_variables/L_demand_coef.txt', 'w') as model_text:
    for listitem in L_model_params:
        model_text.write('%s\n' % listitem)

In [48]:
demand.to_csv("test.csv", index=False)